# 3. Inference trên ảnh mẫu

Notebook này chạy inference trên các ảnh mẫu để test hệ thống VQA.


## 1. Cài đặt thư viện


In [1]:
%pip install -q transformers accelerate bitsandbytes qwen-vl-utils sentence-transformers rank-bm25 underthesea wikipedia pillow pandas tqdm


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 33.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 89.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.4/978.4 kB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 50.7 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [19]:
# from huggingface_hub import login
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# token = user_secrets.get_secret("HF_TOKEN")

# login(token)

## 2. Copy source code vào working directory


In [2]:
import shutil
from pathlib import Path

# Copy source code to Kaggle working directory
if Path('/kaggle/working').exists():
    # Option 1: Copy from input dataset (if you uploaded code as dataset)
    code_source = Path('/kaggle/input') / 'ravqa-code' / 'code'  # Adjust path to your dataset
    code_dest = Path('/kaggle/working/code')
    
    if code_source.exists():
        if code_dest.exists():
            shutil.rmtree(code_dest)
        shutil.copytree(code_source, code_dest)
        print(f" Code copied to {code_dest}")
    else:
        print(f"  Code source not found at {code_source}")
        print("Creating src directory - you'll need to add your source files manually")
        # Create src directory structure manually
        code_dest = Path('/kaggle/working/code/src')
        code_dest.mkdir(parents=True, exist_ok=True)
        print(f"Created {code_dest}")
        print("Please add your source files (pipeline.py, vision.py, etc.) to this directory")
else:
    print("Not running in Kaggle environment")
    code_dest = Path('../src')


 Code copied to /kaggle/working/code


## 3. Tạo source code files (nếu chưa có trong dataset)


In [3]:
# Nếu bạn chưa upload code lên Kaggle dataset, bạn có thể tạo các file cần thiết ở đây
# Hoặc copy từ input dataset nếu đã upload

from pathlib import Path
import os

code_dir = Path('/kaggle/working/code/src') if Path('/kaggle/working').exists() else Path('../src')
code_dir.mkdir(parents=True, exist_ok=True)

# Kiểm tra xem đã có file chưa
if (code_dir / 'pipeline.py').exists():
    print(" Source files already exist")
else:
    print("  Source files not found. Please:")
    print("1. Upload your code folder as a Kaggle dataset, OR")
    print("2. Manually copy pipeline.py, vision.py, answering.py, retrieval.py, config.py to:")
    print(f"   {code_dir}")
    print("\nYou can also use the 'Add Data' feature in Kaggle to upload your code folder.")


 Source files already exist


## 4. Import thư viện và setup paths


In [9]:
pwd

'/kaggle/working'

In [13]:
import json
import sys
from pathlib import Path
from PIL import Image
import pandas as pd
from tqdm import tqdm
import torch
import os

# Add src to path
if Path('/kaggle/working').exists():
    sys.path.insert(0, '/kaggle/working/code/src')
else:
    sys.path.insert(0, '../src')
    
sys.path.insert(0, '/kaggle/working/code')


In [14]:
# Import pipeline
try:
    from src.pipeline import RAGVQAPipeline
    print(" Pipeline imported successfully")
except ImportError as e:
    print(f" Error importing pipeline: {e}")
    print("Please make sure source files are in the correct location")
    raise

2025-12-29 12:56:43.938234: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767013004.148869      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767013004.230391      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767013004.739178      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767013004.739219      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767013004.739222      55 computation_placer.cc:177] computation placer alr

 Pipeline imported successfully


## 5. Cấu hình đường dẫn


In [23]:
# Paths cho Kaggle
if Path('/kaggle/working').exists():
    # Kaggle paths - ĐIỀU CHỈNH THEO DATASET CỦA BẠN
    KB_PATH = "/kaggle/input/knowledge-base/knowledge_base.json"
    IMAGES_DIR = "/kaggle/input/vqa-dataset/images_flat/images_flat"
    VECTOR_DB_PATH = "/kaggle/input/vector-db"  # Nếu đã build index trước đó
    OUTPUT_DIR = "/kaggle/working/results"
else:
    # Local paths
    KB_PATH = "../../data/knowledge_base.json"
    IMAGES_DIR = "../../data/images_flat"
    VECTOR_DB_PATH = "../../models/vector_db"
    OUTPUT_DIR = "../../results"

# Create output directory
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

# Sample images to test - THAY ĐỔI THEO TÊN ẢNH THỰC TẾ
SAMPLE_IMAGES = [
    "004122.jpg",
    "004129.jpg",
    "004140.jpg",
    "004151.jpg",
    "004160.jpg"
]

# Sample questions
SAMPLE_QUESTIONS = [
    "Đây là gì? Hãy mô tả chi tiết hình ảnh này.",
    "Ý nghĩa lịch sử của hình ảnh này là gì?",
    "Bối cảnh văn hóa của hình ảnh này là gì?",
    "Hãy giải thích chi tiết về hình ảnh này.",
    "Thông tin trong hình ảnh này nói về gì?"
]

print(f"Knowledge Base: {KB_PATH}")
print(f"Images Directory: {IMAGES_DIR}")
print(f"Output Directory: {OUTPUT_DIR}")
print(f"\nWill process {len(SAMPLE_IMAGES)} sample images")


Knowledge Base: /kaggle/input/knowledge-base/knowledge_base.json
Images Directory: /kaggle/input/vqa-dataset/images_flat/images_flat
Output Directory: /kaggle/working/results

Will process 5 sample images


## 6. Khởi tạo Pipeline


In [21]:
print("Initializing pipeline...")
print("This may take a few minutes to load the models...")
print("(Downloading Qwen2VL-7B model if not cached)")

# Initialize pipeline with 4-bit quantization for T4 GPU
pipeline = RAGVQAPipeline(
    vision_model_name="Qwen/Qwen2-VL-7B-Instruct",
    answering_model_name="Qwen/Qwen2-VL-7B-Instruct",
    use_4bit=True
)

print(" Pipeline ready!")


Initializing pipeline...
This may take a few minutes to load the models...
(Downloading Qwen2VL-7B model if not cached)


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

 Pipeline ready!


## 7. Chạy inference trên ảnh mẫu


In [24]:
results = []

for i, image_name in enumerate(tqdm(SAMPLE_IMAGES, desc="Processing images")):
    try:
        # Load image
        image_path = Path(IMAGES_DIR) / image_name
        
        if not image_path.exists():
            print(f"  Image not found: {image_path}")
            continue
        
        image = Image.open(image_path).convert('RGB')
        
        # Use corresponding question or default
        question = SAMPLE_QUESTIONS[i] if i < len(SAMPLE_QUESTIONS) else SAMPLE_QUESTIONS[0]
        
        print(f"\n{'='*60}")
        print(f"Image: {image_name}")
        print(f"Question: {question}")
        print(f"{'='*60}")
        
        # Get prediction
        result = pipeline.process(
            image=image,
            question=question,
            return_intermediate=True
        )
        
        # Store results
        results.append({
            'image': image_name,
            'question': question,
            'answer': result['answer'],
            'caption': result.get('caption', ''),
            'ocr': result.get('ocr', ''),
            'num_retrieved': len(result.get('retrieved_docs', [])),
            'context': result.get('context', '')[:500]  # Truncate for display
        })
        
        # Display result
        print(f"\n Answer:")
        print(result['answer'])
        print(f"\n Caption: {result.get('caption', '')[:200]}...")
        print(f"\n OCR: {result.get('ocr', '')[:200] if result.get('ocr') else 'None'}...")
        print(f"\n Retrieved docs: {len(result.get('retrieved_docs', []))}")
        
    except Exception as e:
        print(f" Error processing {image_name}: {e}")
        import traceback
        traceback.print_exc()
        results.append({
            'image': image_name,
            'question': question if 'question' in locals() else '',
            'answer': f'ERROR: {str(e)}',
            'caption': '',
            'ocr': '',
            'num_retrieved': 0,
            'context': ''
        })

print(f"\n Completed inference on {len(results)} images")


Processing images:   0%|          | 0/5 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Image: 004122.jpg
Question: Đây là gì? Hãy mô tả chi tiết hình ảnh này.


ERROR:src.vision:Error in OCR: (1, 'Error opening data file /usr/share/tesseract-ocr/4.00/tessdata/vie.traineddata Please make sure the TESSDATA_PREFIX environment variable is set to your "tessdata" directory. Failed loading language \'vie\' Tesseract couldn\'t load any languages! Could not initialize tesseract.')
Processing images:  20%|██        | 1/5 [00:42<02:50, 42.64s/it]


 Answer:
Hình ảnh này mô tả một cậu bé đang chơi trên một chiếc xe đạp đồ chơi hình con báo. Cậu bé mặc một chiếc áo thun màu đỏ và quần short màu xanh. Trên chiếc xe đạp, có một hình vẽ của con báo với màu vàng và đen, tạo nên một cảm giác vui nhộn và tươi mới. 

Bối cảnh của hình ảnh là một công viên hoặc khu vui chơi trẻ em, với một cây lớn và một cây cầu trong khoảng cách. Cây cầu có nhiều cột và đường đi bộ, tạo nên một cảnh quan mở và thoáng đãng. 

Không có công trình hoặc đặc điểm văn hóa cụ thể được mô tả trong hình ảnh này.

 Caption: Hình ảnh này mô tả một cậu bé đang chơi trên một chiếc xe đạp đồ chơi hình con báo. Cậu bé mặc một chiếc áo thun màu đỏ và quần short màu xanh. Trên chiếc xe đạp, có một hình vẽ của con báo với màu và...

 OCR: None...

 Retrieved docs: 3

Image: 004129.jpg
Question: Ý nghĩa lịch sử của hình ảnh này là gì?


ERROR:src.vision:Error in OCR: (1, 'Error opening data file /usr/share/tesseract-ocr/4.00/tessdata/vie.traineddata Please make sure the TESSDATA_PREFIX environment variable is set to your "tessdata" directory. Failed loading language \'vie\' Tesseract couldn\'t load any languages! Could not initialize tesseract.')
Processing images:  40%|████      | 2/5 [01:16<01:53, 37.74s/it]

...


 Answer:
Bối cảnh văn hóa của hình ảnh này là một lễ hội truyền thống ở Việt Nam. Hình ảnh này mô tả một cảnh lễ hội với các người mặc trang phục truyền thống, chủ yếu là màu đỏ và vàng, với các họa tiết đặc trưng của văn hóa Việt Nam. Họ đang tham gia vào một nghi lễ hoặc lễ hội, với một số người đang chơi trống và một số khác đang chuẩn bị thức ăn. Hình ảnh này liên kết với các lễ hội truyền thống ở Việt Nam, như lễ hội nông nghiệp hoặc lễ hội truyền thống khác.

 Caption: Hình ảnh này mô tả một cảnh lễ hội truyền thống ở Việt Nam. Các người trong hình đang mặc trang phục truyền thống, chủ yếu là màu đỏ và vàng, với các họa tiết đặc trưng của văn hóa Việt Nam. Họ đang t...

 OCR: None...

 Retrieved docs: 3

Image: 004151.jpg
Question: Hãy giải thích chi tiết về hình ảnh này.


ERROR:src.vision:Error in OCR: (1, 'Error opening data file /usr/share/tesseract-ocr/4.00/tessdata/vie.traineddata Please make sure the TESSDATA_PREFIX environment variable is set to your "tessdata" directory. Failed loading language \'vie\' Tesseract couldn\'t load any languages! Could not initialize tesseract.')
Processing images:  80%|████████  | 4/5 [03:17<00:54, 54.65s/it]


 Answer:
Hình ảnh này là một quả xoay, một loại đồ chơi truyền thống phổ biến ở Việt Nam. Đồ chơi này có hình dạng giống như một quả cầu hình cầu, nhưng với một phần trên đầu có hình dạng giống như một nắp lọ. Đồ chơi được làm từ chất liệu trong suốt, có màu xanh lá cây. Đa số các phần của quả xoay đều có màu xanh lá cây, nhưng có một phần trên đầu có màu khác nhau, có thể là màu đỏ hoặc màu khác. Đồ chơi có một đường kính lớn và một đường kính nhỏ hơn ở phần trên đầu. Đa số các phần của quả xoay đều có hình dạng giống như một nắp lọ, nhưng có một phần trên đầu có hình dạng giống như một nắp lọ. Đồ chơi có một đường kính lớn và một đường kính nhỏ hơn ở phần trên đầu. Đa số các phần của quả xoay đều có hình dạng giống như một nắp lọ, nhưng có một phần trên đầu có hình dạng giống như một nắp lọ. Đồ chơi có một đường kính lớn và một đường kính nhỏ hơn ở phần trên đầu. Đa số các phần của quả xoay đều có hình dạng giống như một nắp lọ, nhưng có một phần trên đầu có hình dạng giống như một 

ERROR:src.vision:Error in OCR: (1, 'Error opening data file /usr/share/tesseract-ocr/4.00/tessdata/vie.traineddata Please make sure the TESSDATA_PREFIX environment variable is set to your "tessdata" directory. Failed loading language \'vie\' Tesseract couldn\'t load any languages! Could not initialize tesseract.')
Processing images: 100%|██████████| 5/5 [04:07<00:00, 49.53s/it]


 Answer:
Thông tin trong hình ảnh này nói về Vịnh Hạ Long, một di sản thế giới nổi tiếng ở Việt Nam. Hình ảnh này cho thấy cảnh quan tự nhiên tuyệt đẹp với hàng ngàn hòn đảo và núi đá vôi, tạo nên một cảnh quan độc đáo và huyền ảo. Hình ảnh cũng cho thấy nhiều màu sắc đa dạng, từ màu xanh lá cây của cây cỏ trên các hòn đảo, đến màu xanh ngắt của nước biển và màu trắng của các ngôi nhà nhỏ trên các hòn đảo. Có rất nhiều thuyền và tàu thuyền đang di chuyển trên mặt nước, tạo nên một cảnh quan sinh động và sôi động.

 Caption: Hình ảnh này là một bức ảnh chụp từ trên cao, cho thấy một cảnh quan tự nhiên tuyệt đẹp với hàng ngàn hòn đảo nhỏ và lớn trải dài trên mặt nước. Đây là một khu vực có nhiều hòn đảo và núi đá vôi, tạo ...

 OCR: None...

 Retrieved docs: 3

 Completed inference on 5 images


## 8. Lưu kết quả


In [25]:
# Save to JSON
output_json = f"{OUTPUT_DIR}/inference_results.json"
with open(output_json, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)
print(f" Results saved to {output_json}")

# Save to CSV for easy viewing
df = pd.DataFrame(results)
output_csv = f"{OUTPUT_DIR}/inference_results.csv"
df.to_csv(output_csv, index=False, encoding='utf-8-sig')
print(f" Results saved to {output_csv}")

# Display summary
print("\n=== Inference Summary ===")
print(f"Total images processed: {len(results)}")
if len(results) > 0:
    print(f"Average retrieved docs: {df['num_retrieved'].mean():.2f}")
    print(f"Images with caption: {(df['caption'] != '').sum()}")
    print(f"Images with OCR: {(df['ocr'] != '').sum()}")


 Results saved to /kaggle/working/results/inference_results.json
 Results saved to /kaggle/working/results/inference_results.csv

=== Inference Summary ===
Total images processed: 5
Average retrieved docs: 1.80
Images with caption: 5
Images with OCR: 0


## 9. Hiển thị kết quả chi tiết


In [26]:
# Display all results
for i, result in enumerate(results):
    print(f"\n{'='*80}")
    print(f"Sample {i+1}: {result['image']}")
    print(f"{'='*80}")
    print(f"Question: {result['question']}")
    print(f"\nAnswer:")
    print(result['answer'])
    if result['caption']:
        print(f"\nCaption: {result['caption'][:300]}..." if len(result['caption']) > 300 else f"\nCaption: {result['caption']}")
    if result['ocr']:
        print(f"\nOCR: {result['ocr'][:200]}..." if len(result['ocr']) > 200 else f"\nOCR: {result['ocr']}")
    print(f"\nRetrieved docs: {result['num_retrieved']}")



Sample 1: 004122.jpg
Question: Đây là gì? Hãy mô tả chi tiết hình ảnh này.

Answer:
Hình ảnh này mô tả một cậu bé đang chơi trên một chiếc xe đạp đồ chơi hình con báo. Cậu bé mặc một chiếc áo thun màu đỏ và quần short màu xanh. Trên chiếc xe đạp, có một hình vẽ của con báo với màu vàng và đen, tạo nên một cảm giác vui nhộn và tươi mới. 

Bối cảnh của hình ảnh là một công viên hoặc khu vui chơi trẻ em, với một cây lớn và một cây cầu trong khoảng cách. Cây cầu có nhiều cột và đường đi bộ, tạo nên một cảnh quan mở và thoáng đãng. 

Không có công trình hoặc đặc điểm văn hóa cụ thể được mô tả trong hình ảnh này.

Caption: Hình ảnh này mô tả một cậu bé đang chơi trên một chiếc xe đạp đồ chơi hình con báo. Cậu bé mặc một chiếc áo thun màu đỏ và quần short màu xanh. Trên chiếc xe đạp, có một hình vẽ của con báo với màu vàng và đen, tạo nên một cảm giác vui nhộn và tươi mới. 

Bối cảnh của hình ảnh là một công viên hoặc ...

Retrieved docs: 3

Sample 2: 004129.jpg
Question: Ý nghĩa lịch sử của

## 10. Test với một ảnh và câu hỏi tùy chỉnh


In [27]:
# Test với một ảnh cụ thể - THAY ĐỔI THEO Ý BẠN
test_image_name = "004122.jpg"  # Thay đổi tên ảnh
test_question = "Đây là gì? Hãy mô tả chi tiết và giải thích ý nghĩa lịch sử."

test_image_path = Path(IMAGES_DIR) / test_image_name

if test_image_path.exists():
    test_image = Image.open(test_image_path).convert('RGB')
    
    print(f"Testing with image: {test_image_name}")
    print(f"Question: {test_question}")
    print("\nProcessing...")
    
    result = pipeline.process(
        image=test_image,
        question=test_question,
        return_intermediate=True
    )
    
    print(f"\n{'='*80}")
    print("RESULT:")
    print(f"{'='*80}")
    print(f"\n Answer:")
    print(result['answer'])
    print(f"\n Caption:")
    print(result.get('caption', ''))
    print(f"\n OCR:")
    print(result.get('ocr', 'None'))
    print(f"\n Retrieved Context:")
    print(result.get('context', 'None')[:1000])  # Truncate long context
else:
    print(f"Image not found: {test_image_path}")


Testing with image: 004122.jpg
Question: Đây là gì? Hãy mô tả chi tiết và giải thích ý nghĩa lịch sử.

Processing...


ERROR:src.vision:Error in OCR: (1, 'Error opening data file /usr/share/tesseract-ocr/4.00/tessdata/vie.traineddata Please make sure the TESSDATA_PREFIX environment variable is set to your "tessdata" directory. Failed loading language \'vie\' Tesseract couldn\'t load any languages! Could not initialize tesseract.')



RESULT:

 Answer:
Hình ảnh này mô tả một cậu bé đang chơi trên một chiếc xe đạp đồ chơi hình con báo. Cậu bé mặc một chiếc áo thun màu đỏ và quần short màu xanh. Trên chiếc xe đạp, có một hình vẽ của con báo với màu vàng và đen, tạo nên một cảm giác vui nhộn và tươi mới. 

Bối cảnh của hình ảnh là một công viên hoặc khu vui chơi trẻ em, với một cây lớn và một cây cầu trong khoảng cách. Cây cầu có nhiều cột và đường đi bộ, tạo nên một cảnh quan mở và thoáng đãng. 

Không có công trình hoặc đặc điểm văn hóa cụ thể được mô tả trong hình ảnh này.

 Caption:
Hình ảnh này mô tả một cậu bé đang chơi trên một chiếc xe đạp đồ chơi hình con báo. Cậu bé mặc một chiếc áo thun màu đỏ và quần short màu xanh. Trên chiếc xe đạp, có một hình vẽ của con báo với màu vàng và đen, tạo nên một cảm giác vui nhộn và tươi mới. 

Bối cảnh của hình ảnh là một công viên hoặc khu vui chơi trẻ em, với một cây lớn và một cây cầu trong khoảng cách. Cây cầu có nhiều cột và đường đi bộ, tạo nên một cảnh quan mở và tho